# Harmonizome ETL: Comparative Toxicogenomics Database (CTD) - Gene-Chemical Interactions

Created by: Charles Dai <br>
Credit to: Moshe Silverstein

Data Source: http://ctdbase.org/downloads

In [ ]:
# appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [ ]:
import sys
import os
from datetime import date

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import harmonizome.utility_functions as uf
import harmonizome.lookup as lookup

In [ ]:
%load_ext autoreload
%autoreload 2

### Notebook Information

In [ ]:
print('This notebook was run on:', date.today(), '\nPython version:', sys.version)

# Initialization

In [ ]:
%%appyter hide_code

{% do SectionField(
    name='data',
    title='Upload Data',
    img='load_icon.png'
) %}

In [ ]:
%%appyter code_eval

{% do DescriptionField(
    name='description',
    text='The example below was sourced from <a href="http://ctdbase.org/downloads/" target="_blank">ctdbase.org</a>. If clicking on the example does not work, it should be downloaded directly from the source website.',
    section='data'
) %}

{% set df_file = FileField(
    constraint='.*\.tsv.gz$',
    name='chemical_genes', 
    label='Chemical-Gene Interactions (tsv.gz)', 
    default='Input/CTD/CTD_chem_gene_ixns.tsv.gz',
    examples={
        'CTD_chem_gene_ixns.tsv.gz': 'http://ctdbase.org/reports/CTD_chem_gene_ixns.tsv.gz'
    },
    section='data'
) %}

### Load Mapping Dictionaries

In [ ]:
symbol_lookup, geneid_lookup = lookup.get_lookups()

### Output Path

In [ ]:
output_name = 'ctd_chemical'

path = 'Output/CTD-Chemical'
if not os.path.exists(path):
    os.makedirs(path)

# Load Data

In [ ]:
%%appyter code_exec

df = pd.read_csv(
    {{df_file}}, 
    sep='\t', skiprows=[x for x in range(27)] + [28], 
    usecols=['# ChemicalName', 'GeneSymbol', 'OrganismID', 'GeneForms'])

In [ ]:
df.head()

In [ ]:
df.shape

# Pre-process Data

## Get Relevant Data

In [ ]:
# Get only relevant species
df = df[np.logical_or.reduce([
    df['OrganismID'] == 9606, # human
    df['OrganismID'] == 10090, # mouse
    df['OrganismID'] == 10116 # rat
])]

In [ ]:
# Get only interactions that are gene or protein type
df = df[np.logical_or(
    df['GeneForms'] == 'protein',
    df['GeneForms'] == 'gene'
)]
df.head()

In [ ]:
df = df[['GeneSymbol', '# ChemicalName']]
df.columns = ['Gene Symbol', 'Chemical Name']
df = df.set_index('Gene Symbol')
df.head()

# Filter Data

## Map Gene Symbols to Up-to-date Approved Gene Symbols

In [ ]:
df = uf.map_symbols(df, symbol_lookup, remove_duplicates=True)
df.shape

# Analyze Data

## Create Binary Matrix

In [ ]:
binary_matrix = uf.binary_matrix(df)
binary_matrix.head()

In [ ]:
binary_matrix.shape

In [ ]:
uf.save_data(binary_matrix, path, output_name + '_binary_matrix', 
            compression='npz', dtype=np.uint8)

## Create Gene List

In [ ]:
gene_list = uf.gene_list(binary_matrix, geneid_lookup)
gene_list.head()

In [ ]:
gene_list.shape

In [ ]:
uf.save_data(gene_list, path, output_name + '_gene_list',
            ext='tsv', compression='gzip', index=False)

## Create Attribute List

In [ ]:
attribute_list = uf.attribute_list(binary_matrix)
attribute_list.head()

In [ ]:
attribute_list.shape

In [ ]:
uf.save_data(attribute_list, path, output_name + '_attribute_list',
            ext='tsv', compression='gzip')

## Create Gene and Attribute Set Libraries

In [ ]:
uf.save_setlib(binary_matrix, 'gene', 'up', path, output_name + '_gene_up_set')

In [ ]:
uf.save_setlib(binary_matrix, 'attribute', 'up', path, 
                           output_name + '_attribute_up_set')

## Create Attribute Similarity Matrix

In [ ]:
attribute_similarity_matrix = uf.similarity_matrix(binary_matrix.T, 'jaccard', sparse=True)
attribute_similarity_matrix.head()

In [ ]:
uf.save_data(attribute_similarity_matrix, path,
            output_name + '_attribute_similarity_matrix', 
            compression='npz', symmetric=True, dtype=np.float32)

## Create Gene Similarity Matrix

In [ ]:
gene_similarity_matrix = uf.similarity_matrix(binary_matrix, 'jaccard', sparse=True)
gene_similarity_matrix.head()

In [ ]:
uf.save_data(gene_similarity_matrix, path, 
            output_name + '_gene_similarity_matrix',
            compression='npz', symmetric=True, dtype=np.float32)

## Create Gene-Attribute Edge List

In [ ]:
edge_list = uf.edge_list(binary_matrix)
uf.save_data(edge_list, path, output_name + '_edge_list', 
        ext='tsv', compression='gzip')

# Create Downloadable Save File

In [ ]:
uf.archive(path)

### Link to download output files: [click here](./output_archive.zip)